In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [3]:
df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [4]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [5]:
df["sentiment"]=df["sentiment"].map({'positive':1,'negative':0})

In [6]:
import re

df["review"]=df["review"].apply(lambda x:re.sub(r'<.*?>',' ',x))

In [7]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming t...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [8]:
from sklearn.model_selection import train_test_split

train_x,temp_x,train_y,temp_y=train_test_split(
    df['review'],df['sentiment'],
    test_size=0.2,
    random_state=42)

val_x,test_x,val_y,test_y=train_test_split(
    temp_x,temp_y,
    test_size=0.5,
    random_state=42
)

In [9]:
from transformers import AutoTokenizer
import tensorflow as tf

model_name='bert-base-uncased'

tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

2025-08-10 12:10:27.152825: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754827827.345709      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754827827.404319      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
def tokenize(text,labels):
    
    encodings=tokenizer(
        list(text),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='tf'
    )

    dataset=tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        list(labels)
    ))
    return dataset

In [11]:
train_set=tokenize(train_x,train_y)
test_set=tokenize(test_x,test_y)
val_set=tokenize(val_x,val_y)

train_set=train_set.shuffle(1000).batch(16)
test_set=test_set.batch(16)
val_set=val_set.batch(16)

I0000 00:00:1754827854.761970      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [12]:
from transformers import TFBertForSequenceClassification

model=TFBertForSequenceClassification.from_pretrained(model_name,num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
optimizer=tf.keras.optimizers.Adam(
    learning_rate=3e-5,
    epsilon=1e-8,
)

loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]

model.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [15]:
history=model.fit(
    train_set,
    validation_data=val_set,
    epochs=3
)

Epoch 1/3


I0000 00:00:1754828693.851444      99 service.cc:148] XLA service 0x7ebf145b1680 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754828693.852283      99 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1754828693.925874      99 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1754828694.057565      99 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2500/2500 [==============================] - 721s 267ms/step - loss: 0.3110 - accuracy: 0.8655 - val_loss: 0.2697 - val_accuracy: 0.8922
Epoch 2/3
2500/2500 [==============================] - 662s 265ms/step - loss: 0.1734 - accuracy: 0.9323 - val_loss: 0.2918 - val_accuracy: 0.8926
Epoch 3/3
2500/2500 [==============================] - 662s 265ms/step - loss: 0.0868 - accuracy: 0.9689 - val_loss: 0.4401 - val_accuracy: 0.8902


In [16]:
results=model.evaluate(test_set)

print(f"test loss = {results[0]}")
print(f"test accuracy= {results[1]}")

313/313 [==============================] - 29s 94ms/step - loss: 0.4446 - accuracy: 0.8834
test loss = 0.4446278214454651
test accuracy= 0.883400022983551


In [19]:
def predict_sentiment(text):
    inputs=tokenizer(text,return_tensors='tf',truncation=True,padding=True,max_length=128)
    outputs=model(inputs)
    logits=outputs.logits
    predicted_class=tf.argmax(logits,axis=1).numpy()[0]
    if predicted_class==1 :
        return "Positive"

    else:
        return "Negative"

# Let's test some Tricky Examples!

In [31]:
print(predict_sentiment("Not bad, but not good enough to recommend."))

Negative


In [24]:
print(predict_sentiment("I thought this movie would be great, but it was just okay."))

Negative


In [32]:
print(predict_sentiment("first half felt boring and bad at start, but climax paid off"))

Positive


In [30]:
print(predict_sentiment("The special effects were stunning, although the story lacked depth."))

Negative


In [33]:
model.save_pretrained("./fine_tuned_bert_imdb")
tokenizer.save_pretrained("./fine_tuned_bert_imdb")

('./fine_tuned_bert_imdb/tokenizer_config.json',
 './fine_tuned_bert_imdb/special_tokens_map.json',
 './fine_tuned_bert_imdb/vocab.txt',
 './fine_tuned_bert_imdb/added_tokens.json',
 './fine_tuned_bert_imdb/tokenizer.json')